In [36]:
import csv
import os
from datetime import datetime
from random import shuffle
from collections import defaultdict
import numpy as np
from scipy.special import betaln
from prettytable import PrettyTable
import operator

In [2]:
f = open('dataset_1.csv', 'r')
contents = f.read().split('\r\n')
f.close()

In [3]:
headers = contents[0].split(',')
member_details = {}

for i in xrange(1, len(contents)):
    line = contents[i].split(',')
    if len(line) != len(headers):
        continue
    member_details[line[0]] = {}
    for j in xrange(1, len(headers)):
        member_details[line[0]][headers[j]] = line[j].strip()

# Constructing the graph

We create a graph analogous to the user-item graph, with members corresponding to users and topics corresponding to items. 

##### Step 1
Map member names to user ids.

##### Step 2
Map topic names to item ids.

##### Step 3
Create links between users and items.

In [4]:
userids = {}
itemids = {}
all_topics = set([])

user_count = 0
item_count = 0

## Step 1

In [5]:
for member in member_details:
    user_count += 1
    userids[member] = 'u' + str(user_count)

## Step 2

In [6]:
for member in member_details:
    member_details[member]['Topics'] = member_details[member]['Topics'].split(';')
    for t in member_details[member]['Topics']:
        if t == '':
            continue
        all_topics.add(t)
        
for t in all_topics:
    item_count += 1
    itemids[t] = 'i' + str(item_count)

## Step 3

In [7]:
graph = {}

for u in xrange(user_count):
    graph['u' + str(u + 1)] = []
    
for i in xrange(item_count):
    graph['i' + str(i + 1)] = []
    
for u in member_details:
    for i in member_details[u]['Topics']:
        if i == '':
            continue
        graph[userids[u]].append(itemids[i])
        graph[itemids[i]].append(userids[u])

# Cleaning the graph

We now clean the graph i.e. remove all nodes that are totally disconnected. We will not, for now, attempt the cold start problem.

In [8]:
deleted_nodes = []

for node in graph:
    if len(graph[node]) == 0:
        deleted_nodes.append(node)
        
print 'Removing nodes',
print deleted_nodes

for node in deleted_nodes:
    del(graph[node])

Removing nodes ['u90', 'u92', 'u69', 'u61', 'u64', 'u65', 'u67', 'u25', 'u54', 'u124', 'u122', 'u123', 'u42', 'u41', 'u44', 'u139', 'u131', 'u109', 'u102', 'u100', 'u107', 'u104', 'u105', 'u23', 'u114', 'u116', 'u110', 'u119', 'u18', 'u14', 'u17', 'u80', 'u89', 'u160', 'u161', 'u162', 'u165', 'u168', 'u169', 'u170', 'u147', 'u153', 'u157', 'u156', 'u9', 'u4', 'u6', 'u76']


# Split data into training and test sets (80-20)

In [9]:
training = defaultdict(list)
test = defaultdict(list)

for node in graph:
    if 'u' in node:
        test[node] = []
        n = len(graph[node])
        indices = [x for x in xrange(n)]
        shuffle(indices)
        removed_items = []
        for i in xrange((4*n)/5):
            training[node].append(graph[node][indices[i]])
            training[graph[node][indices[i]]].append(node)
        for i in xrange((4*n)/5, n):
            test[node].append(graph[node][indices[i]])
            test[graph[node][indices[i]]].append(node)

In [10]:
# Verifying if the split has been done correctly or not.

mismatches = 0

for node in training:
    if 'i' in node:
        continue
    if len(training[node]) + len(test[node]) != len(graph[node]):
        mismatches += 1
        
print mismatches

0


# Item-rating map

[No. of users who used item, No. of users who did not use item]

In [19]:
num_users = 0
item_rating_map = {}

for node in training:
    if 'u' in node:
        num_users += 1

for node in training:
    if 'i' in node:
        item_rating_map[node] = [len(training[node]), num_users - len(training[node])]

# Scoring Functions

In [20]:
def PIS(item_pair):
    item1 = item_pair[0]
    item2 = item_pair[1]
    total = 0
    for i in range(0,item_rating_map[item2][0]):
        total += np.exp(betaln(item_rating_map[item1][0]+i,item_rating_map[item1][1]+item_rating_map[item2][1]) -\
                        np.log(item_rating_map[item2][1]+i) - \
                        betaln(1+i, item_rating_map[item2][1]) -\
                        betaln(item_rating_map[item1][0],item_rating_map[item1][1])
                       )
    return total

In [21]:
def PPS(item_pair):
    item1 = item_pair[0]
    item2 = item_pair[1]
    p1 = (item_rating_map[item1][0] + 1) * 1.0 / (item_rating_map[item1][0] + item_rating_map[item1][1] + 1)
    p2 = (item_rating_map[item2][0] + 1) * 1.0 / (item_rating_map[item2][0] + item_rating_map[item2][1] + 1)
    return p1 * p2

In [22]:
def PORS(item_pair):
    item1 = item_pair[0]
    item2 = item_pair[1]
    o1 = (item_rating_map[item1][0] + 1) * 1.0 / (item_rating_map[item1][1] + 1)
    o2 = (item_rating_map[item2][0] + 1) * 1.0 / (item_rating_map[item2][1] + 1)
    return o2 / o1

In [23]:
def score(graph, target_user):
    PIS_values = defaultdict(float)
    PPS_values = defaultdict(float)
    PORS_values = defaultdict(float)
    
    score_PIS = defaultdict(float)
    score_PPS = defaultdict(float)
    score_PORS = defaultdict(float)
    
    for primary_item in graph[target_user]:
        for secondary_user in graph[primary_item]:
            if secondary_user == target_user:
                continue
            for secondary_item in graph[secondary_user]:
                if secondary_item in graph[target_user]:
                    continue
                
                if (primary_item, secondary_item) not in PIS_values:
                    PIS_values[(primary_item, secondary_item)] = PIS((primary_item, secondary_item))
                score_PIS[secondary_item] += PIS_values[(primary_item, secondary_item)]

                if (primary_item, secondary_item) not in PPS_values:
                    PPS_values[(primary_item, secondary_item)] = PPS((primary_item, secondary_item))
                score_PPS[secondary_item] += PPS_values[(primary_item, secondary_item)]
                
                if (primary_item, secondary_item) not in PORS_values:
                    PORS_values[(primary_item, secondary_item)] = PORS((primary_item, secondary_item))
                score_PORS[secondary_item] += PORS_values[(primary_item, secondary_item)]
                
    return score_PIS, score_PPS, score_PORS



In [24]:
def generate_ranking_list(score_list): 
    ranking_list = {} 

    for user in score_list: 
        sorted_score = sorted(score_list[user].items(), key=operator.itemgetter(1), reverse=True)
        ranking_list[user]= [item[0] for item in sorted_score]

    return ranking_list 

In [30]:
score_PIS = {} 
score_PPS = {} 
score_PORS = {}

count = 0
for user in training:
    if user[0] == 'u':
        score_PIS[user], score_PPS[user], score_PORS[user] = score(training, user)
        count += 1
        if count%1==0:
            print count,' Users done'

1  Users done
2  Users done
3  Users done
4  Users done
5  Users done
6  Users done
7  Users done
8  Users done
9  Users done
10  Users done
11  Users done
12  Users done
13  Users done
14  Users done
15  Users done
16  Users done
17  Users done
18  Users done
19  Users done
20  Users done
21  Users done
22  Users done
23  Users done
24  Users done
25  Users done
26  Users done
27  Users done
28  Users done
29  Users done
30  Users done
31  Users done
32  Users done
33  Users done
34  Users done
35  Users done
36  Users done
37  Users done
38  Users done
39  Users done
40  Users done
41  Users done
42  Users done
43  Users done
44  Users done
45  Users done
46  Users done
47  Users done
48  Users done
49  Users done
50  Users done
51  Users done
52  Users done
53  Users done
54  Users done
55  Users done
56  Users done
57  Users done
58  Users done
59  Users done
60  Users done
61  Users done
62  Users done
63  Users done
64  Users done
65  Users done
66  Users done
67  Users done
68  

In [37]:
ranking_PIS = generate_ranking_list(score_PIS)
ranking_PPS = generate_ranking_list(score_PPS)
ranking_PORS = generate_ranking_list(score_PORS)

# Metrics

In [49]:
def p_at_k(k, predictions, lp):
    patk = 0.0
    for user in predictions:
        correct = 0
        for i in range(k):
            if i >= len(predictions[user]):
                break
            if predictions[user][i] in lp[user]:
                correct+=1
        patk += 1.0*correct/k
    return patk/len(predictions.keys())

def MAP(predictions, lp):
    MAP = 0.0
    for user in predictions:
        umap = 0.0
        correct = 0
        for i in range(len(predictions[user])):
            if predictions[user][i] in lp[user]:
                correct += 1
                umap += 1.0*correct/(i+1)
        MAP += umap/max(1,len(lp[user]))
    return MAP/len(predictions.keys())

def MRR(predictions, lp):
    MRR = 0.0
    for user in predictions:
        for i in range(len(predictions[user])):
            if predictions[user][i] in lp[user]:
                MRR += 1.0/(i+1)
                break
    return MRR/len(predictions.keys())

In [50]:
def get_lp(predictions):
    lp = {}
    for user in predictions:
        lp[user] = test[user]
    return lp

In [51]:
scores = {'MAP':defaultdict(float), 'MRR':defaultdict(float), 'P@5':defaultdict(float), 'P@10':defaultdict(float)}
t = PrettyTable(['Method', 'MAP', 'MRR', 'P@5', 'P@10'])

predictions = ranking_PIS
lp = get_lp(predictions)
scores['MAP']['PIS'] = MAP(predictions, lp)
scores['MRR']['PIS'] = MRR(predictions, lp)
scores['P@5']['PIS'] = p_at_k(5, predictions, lp)
scores['P@10']['PIS'] = p_at_k(10, predictions, lp)
t.add_row(['PIS', scores['MAP']['PIS'], scores['MRR']['PIS'], scores['P@5']['PIS'], scores['P@10']['PIS']])

predictions = ranking_PPS
lp = get_lp(predictions)
scores['MAP']['PPS'] = MAP(predictions, lp)
scores['MRR']['PPS'] = MRR(predictions, lp)
scores['P@5']['PPS'] = p_at_k(5, predictions, lp)
scores['P@10']['PPS'] = p_at_k(10, predictions, lp)
t.add_row(['PPS', scores['MAP']['PPS'], scores['MRR']['PPS'], scores['P@5']['PPS'], scores['P@10']['PPS']])

predictions = ranking_PORS
lp = get_lp(predictions)
scores['MAP']['PORS'] = MAP(predictions, lp)
scores['MRR']['PORS'] = MRR(predictions, lp)
scores['P@5']['PORS'] = p_at_k(5, predictions, lp)
scores['P@10']['PORS'] = p_at_k(10, predictions, lp)
t.add_row(['PORS', scores['MAP']['PORS'], scores['MRR']['PORS'], scores['P@5']['PORS'], scores['P@10']['PORS']])


In [52]:
print t

+--------+----------------+----------------+----------------+----------------+
| Method |      MAP       |      MRR       |      P@5       |      P@10      |
+--------+----------------+----------------+----------------+----------------+
|  PIS   | 0.114851291852 | 0.34633438032  | 0.188235294118 | 0.172268907563 |
|  PPS   | 0.107095040271 | 0.328939040055 | 0.181512605042 | 0.16974789916  |
|  PORS  | 0.115401321668 | 0.351382351978 | 0.18487394958  | 0.176470588235 |
+--------+----------------+----------------+----------------+----------------+
